In [1]:
import os

In [2]:
%pwd

'd:\\Python\\MLProjectsPW\\CreditCardFraud1\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Python\\MLProjectsPW\\CreditCardFraud1'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/tejas05in/CreditCardFraud1.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="tejas05in"
os.environ["MLFLOW_TRACKING_PASSWORD"]="9efcb5c7b79d0e949378459b922b1462a80fa413"

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [7]:
from CreditCardFraud.constants import *
from CreditCardFraud.utils.common import read_yaml , create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml (config_filepath)
        self.params = read_yaml (params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.RandomForestClassifier

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir= config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params= params,
            metric_file_name=config.metric_file_name,
            target_column="default payment next month",
            mlflow_uri="https://dagshub.com/tejas05in/CreditCardFraud1.mlflow"
        )
        return model_evaluation_config

In [9]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score , roc_auc_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from CreditCardFraud.utils.common import save_json

In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        accuracy = accuracy_score(actual,pred)
        roc_auc = roc_auc_score(actual,pred)
        return accuracy , roc_auc
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column],axis=1)
        test_y = test_data[self.config.target_column]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualitites = model.predict(test_x)

            (accuracy , roc_auc) = self.eval_metrics(test_y , predicted_qualitites)

            # Saving metrics as local
            scores = {"accuracy score":accuracy, "roc auc score":roc_auc}
            save_json(path = Path(self.config.metric_file_name),data = scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric('accuracy score',accuracy)
            mlflow.log_metric('roc auc score',roc_auc)

            # Model registry does not work with file store
            if tracking_uri_type_store != 'file':

                # Register the model
                # There are other ways to use the model registry , which depends on the use case
                # Please refer to the documentation for more information
                # https://mlflow.org/docs/latest/model-registry.html #api-workflow
                mlflow.sklearn.log_model(model,'model' , registered_model_name="RandomForestClassifier")

            else:
                mlflow.sklearn.log_model(model,'model')


In [12]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2023-08-13 21:37:09,872: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-13 21:37:09,877: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-13 21:37:09,880: INFO: common: creating directory at: artifacts]
[2023-08-13 21:37:09,882: INFO: common: creating directory at: artifacts/model_evaluation]
[2023-08-13 21:37:10,504: INFO: common: json file saved at:{path}]


d:\Python\MLProjectsPW\CreditCardFraud1\env\lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
d:\Python\MLProjectsPW\CreditCardFraud1\env\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'RandomForestClassifier' already exists. Creating a new version of this model...
2023/08/13 21:37:23 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForestClassifier, version 2
Created version '2'